<a href="https://colab.research.google.com/github/nerudxlf/graph_co_authors/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files

In [ ]:
files.upload()

In [3]:
class Node:
    def __init__(self, author_id: str, name: str, faculty: str, department: str, country: str, is_omstu: bool,
                 value_affiliations: int, university: str):
        self.author_id = int(author_id)
        self.name = name
        self.faculty = faculty
        self.department = department.replace('"', '')
        self.country = country
        self.university = university
        self.is_omstu = is_omstu
        self.value_affiliations = value_affiliations

    def __str__(self):
        return f'\t\tid {self.author_id}\n' \
               f'\t\tlabel "{self.name}"\n' \
               f'\t\tfaculty "{self.faculty}"\n' \
               f'\t\tcountry "{self.country}"\n' \
               f'\t\tdepartment "{self.department}"\n' \
               f'\t\tis_omstu "{self.is_omstu}"\n' \
               f'\t\tvalue_affiliations "{self.value_affiliations}"\n' \
               f'\t\tuniversity "{self.university}"\n'

    def __eq__(self, other):
        if isinstance(other, Node):
            return self.author_id == other.author_id
        return NotImplemented

    def __hash__(self):
        return self.author_id

class Edge:
    def __init__(self, id_source: str, id_target: str):
        self.id_source = int(id_source)
        self.id_target = int(id_target)
        self.value = 1

    def __eq__(self, other):
        if isinstance(other, Edge):
            if self.id_source == other.id_source or self.id_source == self.id_target:
                self.value += 1
                return True
            return False
        return NotImplemented

    def __hash__(self):
        return self.id_target

    def __str__(self):
        return f'\t\tsource {self.id_source}\n' \
               f'\t\ttarget {self.id_target}\n' \
               f'\t\tvalue {self.value}'

class Dictionary:
    """
    Класс для работы со словарем и со списком сотрудником
    """
    def __init__(self, path_to_dictionary: str, path_to_list_employee: str):
        self.pd_dict = pd.read_excel(path_to_dictionary)
        self.employee = self.pd_dict["Сотрудник"].to_list()
        self.names = self.pd_dict["names"].to_list()

        self.pd_list_of_employee = pd.read_excel(path_to_list_employee)
        self.name_author = self.pd_list_of_employee['Сотрудник']
        self.work_place = self.pd_list_of_employee['Место работы']

    def get_author_form_dictionary(self, name: str) -> tuple:
        for i in range(len(self.employee)):
            try:
                if self.names[i].find(name.lower()) != -1:
                    return self.employee[i], True
            except AttributeError:
                pass
        return name, False

    def get_author_from_list_of_employees(self, name: str) -> tuple:
        for i in range(len(self.name_author)):
            if self.name_author[i].lower() == name.lower():
                return self.__get_unit_name(self.work_place[i])
        return "None", "None"

    @staticmethod
    def __get_unit_name(unit: str) -> tuple:
        """
        Делит строку с местом работы на кафедру и факультет
        :param unit:
        :return:
        """
        unit_list = unit.split('/')
        if len(unit_list) == 2:
            return unit_list[0], unit_list[1]
        return unit_list[0], "None"

class Article:
    def __init__(self, authors, authors_id, title, year, source_title, value, issue, affiliations,
                 authors_with_affiliations):
        self.authors = authors
        self.authors_id = authors_id
        self.title = title
        self.year = year
        self.source_title = source_title
        self.value = value
        self.issue = issue
        self.affiliations = affiliations
        self.authors_with_affiliations = authors_with_affiliations

    def create_node(self, dictionary: Dictionary) -> list:
        """
        1 Узнаю аффилиацию
        2 Проверяю в словаре
        """
        count = 0
        list_node = []
        list_authors_with_affiliations = self.authors_with_affiliations.split('; ')
        list_affiliations = self.affiliations.split('; ')
        list_author_id = self.authors_id.split(';')
        for author_with_affil in list_authors_with_affiliations:
            name, faculty, university, department, country = "None", "None", "None", "None", "None"
            is_omstu = False
            value_affil = 0
            for affil in list_affiliations:
                # Считаю количество аффилиаций указанных автором и получаю данные для ноды
                # Получаю все аффилиации в статье и в цикле ищу эти аффилиации у автора
                # Если нахожу аффилиацию то записываю ее как основную
                # Если нахожу еще аффилиацию то добавляю в value_afff единицу
                name_author = author_with_affil.split('.,')[0].lower() + '.'
                if author_with_affil.find(affil) != -1 and author_with_affil.find('Omsk State Technical University') != -1:
                    value_affil += 1
                    if value_affil == 2:
                        continue
                    name, bool_value = dictionary.get_author_form_dictionary(name_author)
                    if bool_value:
                        faculty, department = dictionary.get_author_from_list_of_employees(name)
                    else:
                        faculty, department = 'OmSTU', 'OmSTU'
                    country = "Russian Federation"
                    university = "Omsk State Technical University"
                    is_omstu = True
                elif author_with_affil.find(affil) != -1:
                    value_affil += 1
                    if is_omstu:
                        continue
                    name = name_author
                    is_omstu = False
                    department = "None"
                    faculty = "None"
                    split_affil = affil.split(', ')
                    country = split_affil[-1]
                    university = ""
                    for university_get in split_affil:
                        if university_get.find("University") != -1:
                            university = university_get
                            break
                        elif university_get.find("Institute") != -1:
                            university = university_get
                            break
            author_id = list_author_id[count]
            node = Node(author_id, name, faculty, department, country, is_omstu, value_affil, university)
            list_node.append(node)
            count += 1
        return list_node

    def create_edges(self):
        list_edge = []
        for i in self.authors_id.split(';'):
            for j in self.authors_id.split(';'):
                if i == "" or j == "":
                    continue
                if i != j:
                    list_edge.append(Edge(i, j))
        return list_edge

    def __str__(self):
        return f"{self.authors} {self.authors_id} {self.title} {self.year} {self.source_title} {self.value} " \
               f"{self.issue} {self.affiliations} {self.authors_with_affiliations}"

class CreateGraphFromScopus:
    def __init__(self, path_to_scopus_file: str, dictionary: Dictionary):
        self.data_scopus = pd.read_excel(path_to_scopus_file)
        self.dict_data = dictionary

        self.authors = self.data_scopus['Authors'].to_list()
        self.authors_id = self.data_scopus['Author(s) ID'].to_list()
        self.title = self.data_scopus['Title'].to_list()
        self.year = self.data_scopus['Year'].to_list()
        self.source_title = self.data_scopus['Source title'].to_list()
        self.volume = self.data_scopus['Volume'].to_list()
        self.issue = self.data_scopus['Issue'].to_list()
        self.affiliations = self.data_scopus['Affiliations'].to_list()
        self.authors_with_affiliations = self.data_scopus['Authors with affiliations'].to_list()

    def get_articles(self) -> list:
        list_article = []
        for i in range(len(self.authors)):
            article = Article(self.authors[i], self.authors_id[i], self.title[i], self.year[i], self.source_title[i],
                              self.volume[i], self.issue[i], self.affiliations[i], self.authors_with_affiliations[i])
            list_article.append(article)
        return list_article

    def get_nodes_list(self) -> list:
        list_articles = self.get_articles()
        list_nodes = []
        for article in list_articles:
            for node in article.create_node(self.dict_data):
                list_nodes.append(node)
        return list(set(list_nodes))

    def get_edge_list(self) -> list:
        list_articles = self.get_articles()
        list_edge = []
        for article in list_articles:
            for edge in article.create_edges():
                list_edge.append(edge)
        return list(set(list_edge))

    @staticmethod
    def to_gml(list_nodes: list, list_edges: list):
        """
        ФУнкция формирует gml файл из 2 списков объектов
        :param list_nodes: список с нодами
        :param list_edges: список связей нод
        :return:
        """
        str_to_write = "graph\n[\n"
        for node in list_nodes:
            str_to_write += f"\tnode\n\t[\n{node.__str__()}\n\t]\n"
        for edge in list_edges:
            str_to_write += f"\tedge\n\t[\n{edge.__str__()}\n\t]\n"
        str_to_write += "]"
        f = open("graph.gml", "w", encoding="utf-8", )
        f.write(str_to_write)
        f.close()

In [ ]:
dict_data = Dictionary("dictionary.xlsx", "list_of_employees.xlsx")
graph = CreateGraphFromScopus("scopus2020.xlsx", dict_data)
list_node = graph.get_nodes_list()
for node in list_node:
  if node.faculty == "OmSTU" and node.department == "OmSTU":
    print(node)
list_edge = graph.get_edge_list()
graph.to_gml(list_node, list_edge)